In [1]:
import pandas as pd
import numpy as np

# Select state

In [2]:
from ipywidgets import Select  

In [3]:
states = pd.read_csv("state_abbrv.tsv",sep="\t")
state_widget = Select(description='Select a state', options=states.set_index('state').abbreviation.to_dict(),rows=10) # states.to_records(index=False))
state_widget

Select(description='Select a state', options={'Alabama ': 'AL', 'Alaska ': 'AK', 'Arizona ': 'AZ', 'Arkansas '…

# Get the data

In [4]:
import requests

In [5]:
rest_url = 'https://services9.arcgis.com/6Hv9AANartyT7fJW/ArcGIS/rest/services/USCounties_cases_V1/FeatureServer/0/query?where=ST_Abbr%3D%27{}%27&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=Countyname%2CST_Name%2CConfirmedb%2CConfirmed%2CPOP_ESTIMA&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=3819&datumTransformation=&applyVCSProjection=true&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token='.\
            format(state_widget.get_interact_value())

In [6]:
response = requests.get(rest_url)
covid_json = response.json()

# Process data

Get data into a format that can be used by Plotly.

In [7]:
geo_features = []
lat_longs = []
for rec in covid_json['features'] :
    new_rec = {}
    new_rec['properties'] = rec['attributes']
    new_rec['geometry'] = {'type' : 'Polygon' }
    new_rec['geometry']['coordinates'] = rec['geometry']['rings']
    lat_longs.append(new_rec['geometry']['coordinates'][0][0])
    geo_features.append(new_rec)

In [8]:
geos = { "type" : "FeatureCollection", "features" : geo_features}

Calculate center

In [9]:
center = pd.DataFrame(lat_longs,columns=['lon','lat']).mean().to_dict()

Convert to data frame

In [10]:
features = [ x['attributes'] for x in covid_json['features']]


In [11]:
df = pd.DataFrame(features)
df ['density'] = df.Confirmed / df.POP_ESTIMA * 1_000
df

,Countyname,ST_Name,Confirmedb,Confirmed,POP_ESTIMA,density
0,Acadia,Louisiana,167.229458,104,62190,1.672295
1,Allen,Louisiana,289.006054,74,25605,2.890061
2,Ascension,Louisiana,351.321869,438,124672,3.513219
3,Assumption,Louisiana,551.569507,123,22300,5.515695
4,Avoyelles,Louisiana,160.644555,65,40462,1.606446
...,...,...,...,...,...,...
59,Webster,Louisiana,167.534409,65,38798,1.675344
60,West Baton Rouge,Louisiana,287.584667,76,26427,2.875847
61,West Carroll,Louisiana,36.423238,4,10982,0.364232
62,West Feliciana,Louisiana,297.542044,46,15460,2.975420


# Plot

In [12]:
import plotly.express as px
import plotly.graph_objects as go

In [13]:
fig = px.choropleth_mapbox(df,geojson=geos,
                           featureidkey='properties.Countyname',
                           locations='Countyname',
                           color='density',
                           mapbox_style="carto-positron", 
                           center=center, 
                           #opacity=0.5,
                           zoom=5
                          )

In [15]:
go.FigureWidget(fig)

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'featureidkey': 'properties.Countyname',
…